# June 5, 2024: reconstruct whole-brain network from resting state fMRI time series 

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import pickle 
import copy

from scipy import sparse, stats
from scipy.special import gammaln
import glob
from tqdm import tqdm
import ants
from nipype.interfaces import afni
from itertools import combinations, permutations, product


import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import colorcet as cc

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

240609-17:38:03,963 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.6.1


In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

gt.seed_rng(args.SEED)
np.random.seed(args.SEED)

In [3]:

args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

args.unit = 'seswise' # runwise/seswise/subwise : unit sample of dataset
args.denst = 25 # density of fc matrix

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = f'{BASE_path}/roi_results'
RECONST_method = f'lds' #f'normal_dist'
if RECONST_method == '':
    GRAPH_type = f'correlation_graph' 
else: 
    GRAPH_type = f'reconstructed_graph/{RECONST_method}'
ROI_RESULTS_path = f'{ROI_path}/{DESC}/{args.unit}/{GRAPH_type}'
TS_path = f'{ROI_RESULTS_path}/roi_timeseries'
os.system(f'mkdir -p {TS_path}')
GRAPHS_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPHS_path}')

0

In [4]:
def create_dataframe(files):
    def get_ts(file):
        ts = np.loadtxt(file)
        ts = stats.zscore(ts, axis=0, nan_policy='omit')
        return ts        

    dataset_df = []

    for file in tqdm(files):
        ssr = file.split('/')[-1].split('_')
        sub, ses, run = [ssr[i].split('-')[-1] for i in range(3)]
        ts = get_ts(file)
        # fc = get_fc(ts)
        df = pd.DataFrame({
            'sub':[int(sub[-2:])],
            'ses':[int(ses)],
            'run':[int(run)],
            'ssr':[ssr[:-1]],
            'ts':[ts],
        })
        dataset_df.append(df)
        # save_fc(fc, ssr)
        
    dataset_df = pd.concat(dataset_df)
    dataset_df = dataset_df.sort_values(
        by=['sub', 'ses', 'run'],
    ).reset_index(drop=True)
    return dataset_df

In [5]:
files = sorted(glob.glob(f'{ROI_path}/{DESC}/roi_timeseries/*', recursive=True))
ts_df = create_dataframe(files, )

100%|██████████| 116/116 [00:00<00:00, 128.66it/s]


In [6]:
all_cols = ['sub', 'ses', 'run']
cols = []
idx = [i for i, c in enumerate(all_cols) if args.unit[:3] in c][0]
cols = [c for i, c in enumerate(all_cols) if i <= idx]
cols

['sub', 'ses']

In [7]:
def make_ssr(cols, key):
    d = []
    for c, k in zip(cols, key):
        if 'sub' in c:
            d.append(f'{c}-SLC{k:02d}')
        else:
            d.append(f'{c}-{k}')
    return d

def save_ts(ts, ssr):
    file = f'{"_".join(ssr)}_desc-roi-ts.txt'
    file = f'{TS_path}/{file}'
    np.savetxt(file, ts, fmt='%3f')
    return file

def create_timeseries_df(args, dataset_df):
    graphs_df = []
    for key, group in tqdm(dataset_df.groupby(by=cols)):
        key = key if type(key) == tuple else (key,)
        ssr = make_ssr(cols, key)
        ts = np.concatenate(group['ts'].to_list(), axis=0)
        file = save_ts(ts, ssr)
        
        df = pd.DataFrame({})
        for s in ssr:
            c, k = s.split('-')
            df[c] = [k]
        df['ts'] = [ts]
        df['file'] = [file]

        graphs_df.append(df)

    graphs_df = pd.concat(graphs_df).reset_index(drop=True)
    return graphs_df

In [8]:
ts_df = create_timeseries_df(args, ts_df)
ts_df

100%|██████████| 30/30 [00:01<00:00, 15.97it/s]


,sub,ses,ts,file
0,SLC01,1,"[[1.2065445520382525, 0.31730566969513757, 1.1...",/home/govindas/mouse_dataset/roi_results/type-...
1,SLC01,2,"[[1.1127898226126984, -1.3683128014437655, -2....",/home/govindas/mouse_dataset/roi_results/type-...
2,SLC01,3,"[[1.5715444032688004, 0.26980041013504397, -0....",/home/govindas/mouse_dataset/roi_results/type-...
3,SLC02,1,"[[-1.038526870534452, -0.6315943739845857, -0....",/home/govindas/mouse_dataset/roi_results/type-...
4,SLC02,2,"[[-0.051035538312925885, 0.42101396578273237, ...",/home/govindas/mouse_dataset/roi_results/type-...
5,SLC02,3,"[[-1.532879295185952, -1.7599064233427, -0.474...",/home/govindas/mouse_dataset/roi_results/type-...
6,SLC03,1,"[[-0.8268808095005977, -2.224426319236676, -0....",/home/govindas/mouse_dataset/roi_results/type-...
7,SLC03,2,"[[-0.07555837746815192, -0.18872816284331442, ...",/home/govindas/mouse_dataset/roi_results/type-...
8,SLC03,3,"[[-0.07937635795500822, 0.29609192466067075, -...",/home/govindas/mouse_dataset/roi_results/type-...
9,SLC04,1,"[[0.6455322063176996, -2.146681445200838, -4.3...",/home/govindas/mouse_dataset/roi_results/type-...


In [9]:
TS_FILES = sorted(glob.glob(f'{TS_path}/*', recursive=True))
with open(f'{ROI_RESULTS_path}/all_graphs.txt', 'w') as f:
    f.writelines('\n'.join(TS_FILES))

In [10]:
file = TS_FILES[0]
fs = file.split('/')
fs

['',
 'home',
 'govindas',
 'mouse_dataset',
 'roi_results',
 'type-spatial_size-225_symm-True_braindiv-whl_nrois-162',
 'seswise',
 'reconstructed_graph',
 'lds',
 'roi_timeseries',
 'sub-SLC01_ses-1_desc-roi-ts.txt']

In [11]:
os.system((
    f'bash 05a-desc-reconstruct-graph.sh {ROI_RESULTS_path}/all_graphs.txt {RECONST_method} {args.SEED}'
))

inside 
